In [1]:
import pandas as pd
import yahoo_fantasy_api as yfa
from yahoo_oauth import OAuth2
#import yahoofantasy
import os

In [2]:
#Auth
sc = OAuth2(None, None, from_file="YahooAuth.json")

[2025-10-23 09:51:53,996 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2025-10-23 09:51:54,003 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 71041.13283324242
[2025-10-23 09:51:54,003 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN HAS EXPIRED
[2025-10-23 09:51:54,004 DEBUG] [yahoo_oauth.oauth.refresh_access_token] REFRESHING TOKEN


In [4]:
sc.refresh_access_token()

[2025-10-23 09:52:21,396 DEBUG] [yahoo_oauth.oauth.refresh_access_token] REFRESHING TOKEN


{'token_time': 1761227541.3969622,
 'access_token': '4cw6Bjme4lsixJzSqQA0.ZWlE760YCGXGylrqM2iMNBsvuDtUKnaSFV2JZXYO8UvmQGXLu1SDVMUwJKniTUCbzPhPVehixA8e9gCH_y8y4l8hkqIkEq75WEFv0S.6wzRa9BvWsZEG5Vj1pc7lPwkVOiVCH3MLvtN2TaajFd4p5Z0bKSZltg7vbKTHriepnkJPOgjUangIGddlrWz_MgBAUer3tn3lxBJ1VVcta8MOm.LpV6UUh.Prbhhw0i0d.yjdcF4H7JIlhnFyFrFjFNjISa7OIFoA.QLpwDhRMlvw_.SushKlFmAjt8B5FNl5eGXNmkoJ51Oxt8SDTnAyqFi2I6lx5kuGfojZE_uhRBbV0G.cpL8pq1QpnIirbgcoPXk_X48cRGHI3r8o9jFXjsVz0fO27m2Og3MM8S10_DRnofHTAo.DLEfVzcCfLzfFv39IIn_bNi4UDiVhaAGJbZk61qNl76WOWu0TtdQ3Ln8kKoF.yPtgpRI7UHbGR8ahQE46Fpu5wz5eu1namoeBDit7Fd.IdUoH4bSuRM8ZeI044BbVLGXrQOU2NuRp9Jhj7fLJpZ.SUw16jKtasXVizdtetO9akaJo9sXd8QSWQ1BF8eTrIzGDUVLuO_BFXcuK0a_yGxkttvKlTjwuKC1yybKGwUnJyvnLS2UwRkauiq.mafT_ruArApguFhBRCAoFYJkxl_ScFD9jgOqYPjeaWPnZIoxxpGw__5XgKOEFoTyR5tt_CwZBf7IOVNw01dH_zMuDv1vJ7wlFCx0l3rJkSiIxwjDFQQnetx2E4CPQ35hdS4Bgnz3BLY3WyznKrBHYwNmiNjGZMei4L3KpwvxZ.hrOULX39620pMjJLQDS28jOwMDKg5Qj8HEwTpAV1DH8d5fvH6cCrEHugiKmtHJargBkc8ziXm4UdRTx.RUbKfgdfIy6qbBZic

In [74]:
#Get Game object
gm = yfa.Game(sc, 'nba')

In [75]:
leagues = gm.league_ids()
league = gm.to_league(leagues[-1])

In [76]:
#Dict of categories
categories = {
    "5": "FG%",
    "8": "FT%",
    "10": "3PM",
    "12": "PTS",
    "15": "REB",
    "16": "AST",
    "17": "STL",
    "18": "BLK",
    "19": "TO"
}

In [77]:
all_weeks_data = []

# Loop over each week to gather data
for week in range(1, 23):  # Weeks 1 to 23
    # Fetch matchups data for the specific week
    data = league.matchups(week=week)['fantasy_content']['league'][1]['scoreboard']['0']['matchups']
    
    # Iterate over each matchup and team to extract stats
    for matchup_id, matchup_data in data.items():
        # Check that matchup_data is a dictionary
        if not isinstance(matchup_data, dict):
            continue  # Skip non-dictionary entries

        matchup = matchup_data.get('matchup')
        if not matchup:
            continue

        # Go through each team in the matchup, ensure '0' key exists and is a dictionary
        teams = matchup.get('0', {}).get('teams', {})
        if not isinstance(teams, dict):
            continue

        for team_key, team_data in teams.items():
            # Check that team_data is a dictionary and has the expected structure
            if not isinstance(team_data, dict) or 'team' not in team_data:
                continue
            if not isinstance(team_data['team'], list) or len(team_data['team']) < 2:
                continue

            team = team_data['team'][0]  # Access the team list
            team_name = team[2]['name']  # Extract team name

            # Initialize the row with the team name and the week
            team_row = {'Manager': team_name, 'Week': f"{week}"}

            # Fetch team stats for the week
            stats = team_data['team'][1].get('team_stats', {}).get('stats', [])
            
            # Extract relevant stats by stat_id
            for stat in stats:
                stat_id = stat['stat']['stat_id']
                if stat_id in categories:
                    stat_name = categories[stat_id]
                    team_row[stat_name] = stat['stat']['value']

            # Get the games played and remaining games
            games_info = team_data['team'][1].get('team_remaining_games', {}).get('total', {})
            games_played = games_info.get('completed_games', 0)
            remaining_games = games_info.get('remaining_games', 0)
            
            # Calculate total games and format as "games played / total games"
            total_games = games_played + remaining_games
            team_row['Games Played'] = f"{games_played}/{total_games}"

            # Add team row to the list for all weeks
            all_weeks_data.append(team_row)

# Convert the list of all weeks data to a single DataFrame
all_data = pd.DataFrame(all_weeks_data)

In [78]:
all_data

,Manager,Week,FG%,FT%,3PM,PTS,REB,AST,STL,BLK,TO,Games Played
0,Ducksnation,1,.477,.773,8,87,26,19,5,3,10,5/38
1,Fab's Fascinating Team,1,.563,.700,1,26,5,9,2,0,5,1/34
2,GG no re,1,.471,.500,4,21,14,7,5,0,3,2/36
3,Goat Club,1,.528,.880,2,62,21,16,2,2,9,3/36
4,The Good Gordons,1,.714,.000,0,10,6,0,0,1,4,1/33
...,...,...,...,...,...,...,...,...,...,...,...,...
185,Load Management Legends,19,,,,,,,,,,0/36
186,Thomas's Team,19,,,,,,,,,,0/36
187,Bog's Tots,19,,,,,,,,,,0/33
188,Gabriel's Genius Team,19,,,,,,,,,,0/28


In [79]:
all_data.to_excel('output.xlsx',index=False)